In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sympy.utilities.iterables import partitions

/var/folders/kb/ck8v_3xx5j72trkthwkkcvlw0000gn/T/ipykernel_51481/472990437.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [17]:
def list_to_string(arr : list):
    return ''.join([str(elem) for elem in arr])

def string_to_list(s : str):
    return [int(char) for char in s]

In [18]:
N = 2
memo = {}

In [19]:
def murnaghan_nakayama(n : int, lambda_p, sigma_p):
    print("very start")
    lambda_ = string_to_list(lambda_p)
    sigma = string_to_list(sigma_p)
    print(f"lambda: {lambda_}")
    print(f"sigma: {sigma}")
    
    # Base Cases
    if n == 1:
        print(f'returning: {1}')
        return 1
    
    if n < 1:
        print(f'returning: {0}')
        return 0
    
    # DP
    if (n, lambda_p, sigma_p) in memo:
        return memo[(n, lambda_p, sigma_p)]
    # Get our hooks
    hooks = [(x,y) for x in range(len(lambda_)) for y in range(len(lambda_)) if (lambda_[x] == 0 and lambda_[y] == 1)]

    # Invalid diagram -- return 0
    if len(hooks) == 0:
        memo[(n, lambda_p, sigma_p)] = 0
        print(f'returning: {0}')
        return memo[(n, lambda_p, sigma_p)]

    cycle_lengths = {} # dictionary of unique cycle lengths where key is cycle length and value is starting 0 and ending 1
    curr = 0
    j = 0
    for i in range(len(sigma)):
        if sigma[i] == 0:
            curr+=1
        if sigma[i] == 1:
            cycle_lengths[curr] = (j, i)
            j = i + 1

    print(f'cycle lengths: {cycle_lengths}')
    sum = 0
    # Loop through our hooks
    print(f"hooks: {hooks}")
    for (i, j) in hooks:
        t = j-i
        print(f"t: {t}")
        if t in cycle_lengths: # there exists rho
            # Now need to find rho and tau
            # Just now need to remove the actual row
            tau = sigma[:cycle_lengths[t][0]]+sigma[cycle_lengths[t][1]+1:]
            print(f"tau : {tau}")
            height = lambda_[i:j].count(1)
            erased_bs = lambda_.copy()
            erased_bs[i] = lambda_[j]
            erased_bs[j] = lambda_[i]

            # Remove 1's from start of list
            while len(erased_bs) > 0:
                if erased_bs[0] != 0:
                    del erased_bs[0]
                else:
                    break
        
            # Remove 0's from end of list
            print(f"erased bs: {erased_bs}")
            while len(erased_bs) > 0:
                if erased_bs[-1] != 1:
                    del erased_bs[-1]
                else:
                    break
                
            print(f"erased bs : {erased_bs}")
            sum += ((-1)**height)*murnaghan_nakayama(n-t, list_to_string(erased_bs), list_to_string(tau))
            

    # Done!
    memo[(n, lambda_p, sigma_p)] = sum
    print(f'returning: {sum}')
    return memo[(n, lambda_p, sigma_p)]




In [20]:
# Get list of partitions
parts = list(partitions(N))
bit_strings = []
char_table = []
for lambda_ in parts:
    # Iterate through each partition

    # Make our bit string
    arr = []
    prev = 0
    lamda_prime = reversed(list(lambda_.keys()))
    for key in list(lamda_prime):
        for _ in range(key-prev):
            arr.append(0)
        for _ in range(lambda_[key]):
            arr.append(1)
        prev = key
        
    bit_strings.append(list_to_string(arr))
i = 0
for lambda_ in bit_strings:
    char_table.append([])
    for sigma in bit_strings:
        char_table[i].append(murnaghan_nakayama(N, lambda_, sigma))
    i += 1

print(np.matrix(char_table))


very start
lambda: [0, 0, 1]
sigma: [0, 0, 1]
[(0, 2), (1, 2)]
cycle lengths: {2: (0, 2)}
hooks: [(0, 2), (1, 2)]
tau : []
erased bs: [0, 0]
erased bs : []
very start
lambda: []
sigma: []
returning: 0
returning: 0
very start
lambda: [0, 0, 1]
sigma: [0, 1, 1]
[(0, 2), (1, 2)]
cycle lengths: {1: (2, 2)}
hooks: [(0, 2), (1, 2)]
tau : [0, 1]
erased bs: [0, 1, 0]
erased bs : [0, 1]
very start
lambda: [0, 1]
sigma: [0, 1]
returning: 1
returning: 1
very start
lambda: [0, 1, 1]
sigma: [0, 0, 1]
[(0, 1), (0, 2)]
cycle lengths: {2: (0, 2)}
hooks: [(0, 1), (0, 2)]
tau : []
erased bs: [0]
erased bs : []
very start
lambda: []
sigma: []
returning: 0
returning: 0
very start
lambda: [0, 1, 1]
sigma: [0, 1, 1]
[(0, 1), (0, 2)]
cycle lengths: {1: (2, 2)}
hooks: [(0, 1), (0, 2)]
tau : [0, 1]
erased bs: [0, 1]
erased bs : [0, 1]
very start
lambda: [0, 1]
sigma: [0, 1]
returning: 1
returning: 1
[[0 1]
 [0 1]]
